### 📄 Documentação: Extrator de Composição IBRX-100 (API B3)

### 1. Visão Geral

Este script é um componente de **Extração (E)** dentro do pipeline ETL do projeto Aurum. Sua função é conectar-se diretamente à API "oculta" da B3 (Brasil, Bolsa, Balcão) para obter a carteira teórica atualizada do índice **IBRX-100**.

Diferente de abordagens tradicionais que utilizam *web scraping* (Selenium/BeautifulSoup), este script utiliza engenharia reversa da chamada de API da B3, codificando parâmetros em Base64 para simular uma requisição legítima do navegador.



### 2. Dependências Técnicas

Para executar este script, o ambiente Python deve conter as seguintes bibliotecas:

* **pandas:** Manipulação e estruturação dos dados.
* **requests:** Realização de chamadas HTTP à API.
* **urllib3:** Gerenciamento de conexões e supressão de avisos SSL.
* **Bibliotecas Padrão:** `json`, `base64`, `logging`, `pathlib`, `time`.

### 3. Estrutura do Script

O script não utiliza Classes (POO), mas sim uma arquitetura funcional modular. Abaixo estão as descrições das principais funções:

### **fetch_ibrx100_from_b3_api()**

Esta é a função "core" do extrator.

1. **Prepara Parâmetros:** Define um dicionário JSON com o índice alvo (`IBXX` para IBRX-100) e paginação.
2. **Codificação Base64:** Transforma o JSON em uma string Base64, replicando o comportamento do frontend da B3.
3. **Requisição:** Envia um GET para o endpoint `indexProxy/indexCall/GetPortfolioDay`.
4. **Tratamento de Dados:**
    * Consome o JSON de resposta.
    * **Autocorreção de Colunas:** Verifica dinamicamente o nome da coluna de código do ativo (`codNeg`, `cod`, `acronym`), garantindo robustez caso a B3 altere a API.
    * **Normalização:** Remove espaços em branco e cria a coluna compatível com o Yahoo Finance (sufixo `.SA`).



### `save_data(df)`

Responsável pela persistência dos dados.

1. Verifica se o diretório `../data` existe; caso contrário, cria-o.
2. Salva o DataFrame em dois formatos:
    * **.CSV:** Para inspeção humana e compatibilidade simples.
    * **.PARQUET:** Para leitura de alta performance nos próximos passos do pipeline.

## 4. Dicionário de Dados (Output)

O script gera um dataset padronizado. A tabela abaixo descreve o esquema (schema) do arquivo gerado (`tickers_ibrx100_full`).

| Nome da Coluna | Tipo de Dado (Pandas) | Descrição | Exemplo |
| :--- | :--- | :--- | :--- |
| **`ticker`** | `object` (string) | O código de negociação oficial fornecido pela B3. | `PETR4` |
| **`participacao`** | `object` / `float` | (Opcional) A porcentagem de participação do ativo no índice, se retornada pela API. | `4,520` |
| **`Ticker_Yahoo`** | `object` (string) | O ticker normalizado com o sufixo `.SA`, pronto para consumo pela biblioteca `yfinance`. | `PETR4.SA` |

## 5. Como Executar

Execute o script diretamente via terminal:

```bash
python b3_api_extractor.py

In [2]:
import pandas as pd
import requests
import base64
import json
import logging
from pathlib import Path
import time
import urllib3

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

OUTPUT_DIR = Path("..", "data") 
OUTPUT_FILENAME_CSV = "tickers_ibrx100_full.csv"
OUTPUT_FILENAME_PARQUET = "tickers_ibrx100_full.parquet"

def fetch_ibrx100_from_b3_api() -> pd.DataFrame:
    """
    Consome diretamente a API JSON da B3 para obter a composição do IBRX-100.
    """
    logger.info("Iniciando requisição à API da B3 (IndexProxy)...")
    
    try:
        params = {
            "language": "pt-br",
            "pageNumber": 1,
            "pageSize": 120, 
            "index": "IBXX", 
            "segment": "1"
        }
        
        params_json = json.dumps(params)
        params_b64 = base64.b64encode(params_json.encode("utf-8")).decode("utf-8")
        
        url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{params_b64}"
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(url, headers=headers, timeout=15, verify=False)
        
        if response.status_code != 200:
            logger.error(f"Erro na requisição: Status {response.status_code}")
            return None
            
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            logger.warning("JSON retornado pela B3 está vazio na chave 'results'.")
            return None
            
        logger.info(f"API retornou {len(results)} ativos.")
        
        df = pd.DataFrame(results)
        
        logger.info(f"Colunas encontradas no JSON: {df.columns.tolist()}")
        
        coluna_ticker = None
        possiveis_nomes = ['codNeg', 'cod', 'acronym', 'symbol', 'identifier']
        
        for col in possiveis_nomes:
            if col in df.columns:
                coluna_ticker = col
                logger.info(f"Coluna de ticker identificada como: '{col}'")
                break
        
        if not coluna_ticker:
            logger.error("Não foi possível identificar a coluna de Ticker no DataFrame.")
            logger.error(f"Colunas disponíveis: {df.columns.tolist()}")
            return None

        coluna_part = 'part' if 'part' in df.columns else None
        
        colunas_selecao = [coluna_ticker]
        if coluna_part:
            colunas_selecao.append(coluna_part)
            
        df_final = df[colunas_selecao].copy()
        
        rename_map = {coluna_ticker: 'ticker'}
        if coluna_part:
            rename_map[coluna_part] = 'participacao'
            
        df_final = df_final.rename(columns=rename_map)
        
        logger.info("Normalizando tickers (adicionando .SA)...")
        df_final['ticker'] = df_final['ticker'].str.strip()
        df_final['Ticker_Yahoo'] = df_final['ticker'].apply(lambda x: f"{x}.SA")
        
        return df_final

    except Exception as e:
        logger.error(f"Falha crítica no extrator da API B3: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def save_data(df: pd.DataFrame):
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    path_csv = OUTPUT_DIR / OUTPUT_FILENAME_CSV
    path_parquet = OUTPUT_DIR / OUTPUT_FILENAME_PARQUET
    
    df.to_csv(path_csv, index=False, encoding='utf-8-sig')
    df.to_parquet(path_parquet, index=False)
    
    logger.info(f"💾 Arquivos salvos:")
    logger.info(f"   -> {path_csv}")
    logger.info(f"   -> {path_parquet}")

if __name__ == "__main__":
    start_time = time.time()
    
    df_result = fetch_ibrx100_from_b3_api()
    
    if df_result is not None and not df_result.empty:
        print("\n--- Amostra do IBRX-100 (API B3) ---")
        display(df_result.head())
        save_data(df_result)
    else:
        logger.error("Não foi possível gerar a lista de tickers.")
        
    print(f"\nTempo total: {time.time() - start_time:.2f} segundos")

2025-12-28 20:37:56,206 - INFO - Iniciando requisição à API da B3 (IndexProxy)...
2025-12-28 20:46:40,923 - INFO - API retornou 97 ativos.
2025-12-28 20:46:40,996 - INFO - Colunas encontradas no JSON: ['segment', 'cod', 'asset', 'type', 'part', 'partAcum', 'theoricalQty']
2025-12-28 20:46:41,005 - INFO - Coluna de ticker identificada como: 'cod'
2025-12-28 20:46:41,035 - INFO - Normalizando tickers (adicionando .SA)...



--- Amostra do IBRX-100 (API B3) ---


,ticker,participacao,Ticker_Yahoo
0,ALOS3,"0,529",ALOS3.SA
1,ABEV3,"2,350",ABEV3.SA
2,ANIM3,"0,034",ANIM3.SA
3,ASAI3,"0,391",ASAI3.SA
4,AURE3,"0,152",AURE3.SA


2025-12-28 20:46:41,113 - INFO - 💾 Arquivos salvos:
2025-12-28 20:46:41,115 - INFO -    -> ..\data\tickers_ibrx100_full.csv
2025-12-28 20:46:41,118 - INFO -    -> ..\data\tickers_ibrx100_full.parquet



Tempo total: 524.91 segundos
